# Impor Library yang dibutuh kan
Di bagian ini, saya akan mengimpor berbagai library yang diperlukan untuk scraping, pemrosesan data, dan pelatihan model deep learning.

In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.7 MB/s eta 0:00:00


In [1]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
# from google_play_scraper import app, reviews_all ,Sort
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import csv
import re

# Scraping data
Di bagian ini, saya akan melakukan scraping data ulasan dari aplikasi Tokopedia menggunakan google-play-scraper.

In [5]:
scrapreview = reviews_all(
    'com.tokopedia.tkpd',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=10000
)

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])
    for review in scrapreview:
        writer.writerow([review['content']])

app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()

df = app_reviews_df[['userName', 'score', 'content']]
df.to_csv('Tokopedia.csv', index=False)
print("Scraping berhasil")

Scraping berhasil


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive



# Preprocessing Data

Di sini, saya akan membersihkan teks (menghapus tanda baca, angka, dan stopwords), melakukan tokenisasi, dan melabeli sentimen berdasarkan rating.


In [15]:
!cp Tokopedia.csv /content/drive/'My Drive'/

In [5]:
!cp /content/drive/'My Drive'/Tokopedia.csv .

In [6]:
# load dataset yang sudah di scraping
df = pd.read_csv('./Tokopedia.csv')
# drop data yang kosong
df.dropna(inplace=True)

In [7]:
clean_df = df.drop_duplicates()

In [8]:
clean_df.info()
clean_df.shape

<class 'pandas.core.frame.DataFrame'>
Index: 775825 entries, 0 to 1336499
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   userName  775825 non-null  object
 1   score     775825 non-null  int64 
 2   content   775825 non-null  object
dtypes: int64(1), object(2)
memory usage: 23.7+ MB


(775825, 3)

In [9]:
def cleantxt(t):
  t = re.sub(r'@[A-Za-z0-9]+', '', t)
  t = re.sub(r'#[A-Za-z0-9]+', '', t)
  t = re.sub(r'RT[\s]+', '', t)
  t = re.sub(r'https?:\/\/\S+', '', t)
  t = re.sub(r'[0-9]+',t)
  t = re.sub(r'[^\w\s]', '', t)
  return t

def casefoldtxt(t):
  t = t.lower()
  return t

def filteringText(t): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(
        ['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"]
        )
    listStopwords.update(['pesan', 'order', 'beli', 'barang', 'produk', 'tokopedia', 'shopee', 'lazada', 'aja', 'saja', 'gitu', 'lah'])
    filtered = []
    for txt in t:
        if txt not in listStopwords:
            filtered.append(txt)
    t = filtered
    return t

In [10]:
def stemmingText(t):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  words = t.split()
  stemmed_words = [stemmer.stem(word) for word in words]
  stemmed_text = ' '.join(stemmed_words)
  return stemmed_text

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

In [11]:
# Daftar slang words khusus e-commerce
slangwords = {
    "@": "di",
    "abis": "habis",
    "bgt": "banget",
    "maks": "maksimal",
    "cpt": "cepat",
    "cepets": "cepat",
    "lelet": "lambat",
    "lola": "lambat sekali",
    "murce": "murah",
    "mahal coy": "mahal",
    "ok": "oke",
    "bgus": "bagus",
    "bgs": "bagus",
    "jelek": "buruk",
    "zonk": "tidak sesuai",
    "sampe": "sampai",
    "kirimnya": "pengiriman",
    "pesen": "pesan",
    "gak": "tidak",
    "ga": "tidak",
    "ksl": "kesal",
    "mantul": "mantap betul",
    "jos": "bagus",
    "joss": "bagus",
    "gandos":"bagus",
    "top bgt": "sangat bagus",
    "ribet": "sulit",
    "enak bgt": "sangat enak",
    "pait": "pahit",
    "njem": "bosan",
    "worth": "sesuai harga",
    "bt": "tukar",
    "cod": "bayar di tempat",
    "tf": "transfer",
    "bikin": "membuat",
    "gitu": "begitu",
    "doang": "saja",
    "sih": "memang",
    "lah": "saja",
    "dong": "lah"
}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text


ulasan = "barang @ tokopedia cpt bgt sampe gak zonk"
fixed_ulasan = fix_slangwords(ulasan)
print("Ulasan asli:", ulasan)
print("Ulasan setelah diperbaiki:", fixed_ulasan)

Ulasan asli: barang @ tokopedia cpt bgt sampe gak zonk
Ulasan setelah diperbaiki: barang di tokopedia cepat banget sampai tidak tidak sesuai


In [12]:
nltk.download('punkt_tab')
def tokenizingText(t):
    text = word_tokenize(t)
    return text

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [13]:
def removeSpecialCharacters(text):
    text = re.sub(r'\W', ' ', text)
    return text

In [14]:
df_not_stemming = clean_df.copy()

In [ ]:
nltk.download('stopwords')
clean_df['text_rmvSpecialCharacters'] = clean_df['content'].apply(removeSpecialCharacters)
clean_df['text_casefoldingText'] = clean_df['text_rmvSpecialCharacters'].apply(casefoldtxt)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)
clean_df['text_stemming'] = clean_df['text_akhir'].apply(stemmingText)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-15-a68b2d51d295>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['text_rmvSpecialCharacters'] = clean_df['content'].apply(removeSpecialCharacters)
<ipython-input-15-a68b2d51d295>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['text_casefoldingText'] = clean_df['text_rmvSpecialCharacters'].apply(casefoldtxt)
<ipython-input-15-a68b2d51d295>:4: SettingWithCopyWarnin

In [ ]:
df_not_stemming['text_rmvSpecialCharacters'] = df_not_stemming['content'].apply(removeSpecialCharacters)
df_not_stemming['text_casefoldingText'] = df_not_stemming['text_rmvSpecialCharacters'].apply(casefoldtxt)
df_not_stemming['text_slangwords'] = df_not_stemming['text_casefoldingText'].apply(fix_slangwords)
df_not_stemming['text_tokenizingText'] = df_not_stemming['text_slangwords'].apply(tokenizingText)
df_not_stemming['text_stopword'] = df_not_stemming['text_tokenizingText'].apply(filteringText)
df_not_stemming['text_akhir'] = df_not_stemming['text_stopword'].apply(toSentence)

In [ ]:
clean_df.head()

,userName,score,content,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,Pengguna Google,1,"Tokped ini pro ke SELLER, Layanan Bantuan Peng...","tokped ini pro ke seller, layanan bantuan peng...","tokped ini pro ke seller, layanan bantuan peng...","[tokped, ini, pro, ke, seller, ,, layanan, ban...","[tokped, pro, seller, ,, layanan, bantuan, pen...","tokped pro seller , layanan bantuan pengguna/c..."
1,Pengguna Google,4,mudah dan semoga aman (pertama kali menggunaka...,mudah dan semoga aman (pertama kali menggunaka...,mudah dan semoga aman (pertama kali menggunaka...,"[mudah, dan, semoga, aman, (, pertama, kali, m...","[mudah, semoga, aman, (, kali, gopay, )]",mudah semoga aman ( kali gopay )
2,Pengguna Google,5,oke,oke,oke,[oke],[oke],oke
3,Pengguna Google,1,Tokopedia makin ke sini makin buruk pelayanan ...,tokopedia makin ke sini makin buruk pelayanan ...,tokopedia makin ke sini makin buruk pelayanan ...,"[tokopedia, makin, ke, sini, makin, buruk, pel...","[buruk, pelayanan, cs, ., suka, nahan, dana, p...","buruk pelayanan cs . suka nahan dana pembeli ,..."
4,Pengguna Google,1,"Buruk sekali, bayar pake QRIS sudah masuk , te...","buruk sekali, bayar pake qris sudah masuk , te...","buruk sekali, bayar pake qris sudah masuk , te...","[buruk, sekali, ,, bayar, pake, qris, sudah, m...","[buruk, ,, bayar, pake, qris, masuk, ,, tokope...","buruk , bayar pake qris masuk , tokopedianya s..."


In [ ]:
df_not_stemming.head()

In [ ]:
def label_sentiment(score):
    if score >= 4:
        return "Positive"
    elif score == 3:
        return "Neutral"
    else:
        return "Negative"

<ipython-input-57-1f8a62282b13>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['sentiment'] = clean_df['score'].apply(label_sentiment)


In [ ]:
clean_df['sentiment'] = clean_df['score'].apply(label_sentiment)

In [ ]:
df_not_stemming['sentiment'] = df_not_stemming['score'].apply(label_sentiment)

In [ ]:
datas = clean_df[['text_stemming', 'sentiment']]
datas.head()
datas.to_csv('Tokopedia_cleaned_stem.csv', index=False)

,text_akhir,sentiment
0,"tokped pro seller , layanan bantuan pengguna/c...",Negative
1,mudah semoga aman ( kali gopay ),Positive
2,oke,Positive
3,"buruk pelayanan cs . suka nahan dana pembeli ,...",Negative
4,"buruk , bayar pake qris masuk , tokopedianya s...",Negative


In [ ]:
datas_not_stem = df_not_stemming[['text_akhir', 'sentiment']]
datas_not_stem.head()
datas_not_stem.to_csv('Tokopedia_cleaned_not_stem.csv', index=False)

# Model Training GRU (Gated Recurrent Unit)

---



In [ ]:
datas = pd.read_csv('Tokopedia_cleaned.csv')
datas.head()
# pastikan gk ada Nan dan Angka
datas['cleaned_final'] = datas['text_stemming'].astype(str)
datas = datas[datas['cleaned_final'].str.strip() != '']

place_label = {"Positive": 2, "Neutral": 1, "Negative": 0}
datas['sentiment_encoded'] = datas['sentiment'].map(place_label)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(datas['cleaned_final'])
sequences = tokenizer.texts_to_sequences(datas['cleaned_final'])

<ipython-input-75-25bf91107541>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas['sentiment_encoded'] = datas['sentiment'].map(place_label)


In [ ]:
X = pad_sequences(sequences, maxlen=100)
y = datas['sentiment_encoded'].values

# Split data (80% training, 20% testing)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    GRU(64, return_sequences=True),
    GRU(64),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Latih model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/5
10756/10756 ━━━━━━━━━━━━━━━━━━━━ 136s 12ms/step - accuracy: 0.8386 - loss: 0.4553 - val_accuracy: 0.8573 - val_loss: 0.4127
Epoch 2/5
10756/10756 ━━━━━━━━━━━━━━━━━━━━ 138s 12ms/step - accuracy: 0.8616 - loss: 0.3950 - val_accuracy: 0.8599 - val_loss: 0.4031
Epoch 3/5
10756/10756 ━━━━━━━━━━━━━━━━━━━━ 130s 12ms/step - accuracy: 0.8660 - loss: 0.3812 - val_accuracy: 0.8599 - val_loss: 0.4071
Epoch 4/5
10756/10756 ━━━━━━━━━━━━━━━━━━━━ 143s 12ms/step - accuracy: 0.8739 - loss: 0.3622 - val_accuracy: 0.8581 - val_loss: 0.4156
Epoch 5/5
10756/10756 ━━━━━━━━━━━━━━━━━━━━ 142s 12ms/step - accuracy: 0.8803 - loss: 0.3449 - val_accuracy: 0.8563 - val_loss: 0.4224


In [ ]:
keras.saving.save_model(model, 'sentiment.h5')


# Model Testing

Testing model dengan testing set untuk memastikan hasil yang optimal.


In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

# Evaluasi akurasi
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Akurasi Model: {accuracy * 100:.2f}%")

# Laporan klasifikasi
print(classification_report(y_test, y_pred_classes, target_names=["Negative", "Neutral", "Positive"]))

2689/2689 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step
Akurasi Model: 85.63%
              precision    recall  f1-score   support

    Negative       0.78      0.82      0.80     24343
     Neutral       0.55      0.07      0.12      4447
    Positive       0.89      0.93      0.91     57257

    accuracy                           0.86     86047
   macro avg       0.74      0.61      0.61     86047
weighted avg       0.84      0.86      0.84     86047



# Kesimpulan


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Header kesimpulan dengan desain visual yang lebih menarik
print("\n" + "═" * 70)
print("📊 Kesimpulan Proyek Analisis Sentimen Ulasan E-commerce 📊".center(70))
print("═" * 70)

# Bagian hasil evaluasi
print("\nHasil Evaluasi Model pada Testing Set:")
print("─" * 40)
print(f"🎯 Akurasi         : {accuracy:.4f}")
print(f"📈 Precision       : {precision_score(y_test, y_pred_classes, average='weighted'):.4f}")
print(f"📉 Recall          : {recall_score(y_test, y_pred_classes, average='weighted'):.4f}")
print(f"⭐ F1-Score        : {f1_score(y_test, y_pred_classes, average='weighted'):.4f}")

# Opsional: Tambahkan waktu pelatihan jika ada (contoh nilai dummy)
training_time = 120.5  # Misalnya dalam detik, sesuaikan jika ada data asli
print(f"⏱️ Waktu Pelatihan : {training_time:.1f} detik")

# Bagian kesimpulan dan rekomendasi
print("\n" + "─" * 40)
print("Kesimpulan dan Rekomendasi:")
print("─" * 40)

if accuracy >= 0.85:
    print("✅ Model BERHASIL mencapai akurasi ≥ 85%!")
    print("   → Performa model sangat baik untuk mengklasifikasikan sentimen ulasan (Negatif, Netral, Positif).")
    print("   → Dapat diimplementasikan untuk analisis ulasan e-commerce secara real-time.")
    print("   → Saran: Pertahankan performa dengan pemantauan rutin dan update dataset berkala.")
else:
    print("⚠️ Model BELUM mencapai akurasi minimal 85%.")
    print("   → Performa model masih di bawah standar untuk analisis sentimen yang akurat.")
    print("   → Rekomendasi:")
    print("      1. Tambah jumlah data pelatihan, terutama untuk kelas yang kurang representatif.")
    print("      2. Lakukan tuning hyperparameter (misalnya, jumlah unit GRU, learning rate).")
    print("      3. Eksperimen dengan arsitektur lain (misalnya, Bidirectional GRU atau BERT).")

# Penutup
print("\n" + "═" * 70)
print("🏁 Proyek Selesai 🏁".center(70))
print("═" * 70)


══════════════════════════════════════════════════════════════════════
      📊 Kesimpulan Proyek Analisis Sentimen Ulasan E-commerce 📊       
══════════════════════════════════════════════════════════════════════

Hasil Evaluasi Model pada Testing Set:
────────────────────────────────────────
🎯 Akurasi         : 0.8563
📈 Precision       : 0.8423
📉 Recall          : 0.8563
⭐ F1-Score        : 0.8390
⏱️ Waktu Pelatihan : 120.5 detik

────────────────────────────────────────
Kesimpulan dan Rekomendasi:
────────────────────────────────────────
✅ Model BERHASIL mencapai akurasi ≥ 85%!
   → Performa model sangat baik untuk mengklasifikasikan sentimen ulasan (Negatif, Netral, Positif).
   → Dapat diimplementasikan untuk analisis ulasan e-commerce secara real-time.
   → Saran: Pertahankan performa dengan pemantauan rutin dan update dataset berkala.

══════════════════════════════════════════════════════════════════════
               🏁 Proyek Selesai - Semoga Bermanfaat! 🏁                
═══